### 第3章: 正規表現
 On this page
20.JSONデータの読み込み
21.カテゴリ名を含む行を抽出
22.カテゴリ名の抽出
23.セクション構造
24.ファイル参照の抽出
25.テンプレートの抽出
26.強調マークアップの除去
27.内部リンクの除去
28.MediaWikiマークアップの除去
29.国旗画像のURLを取得する
Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．

1行に1記事の情報がJSON形式で格納される
各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
ファイル全体はgzipで圧縮される
以下の処理を行うプログラムを作成せよ．

### 20.JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [2]:
path = '/Users/seiya.k/workspace/100knock-2020/SeiyaKikuchi/chapter03/jawiki-country.json.gz'

[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]

In [3]:
import json
import gzip
with gzip.open(path) as f, open('article_UK', mode='w') as new_f:
    for l in f:
        l_loaded = json.loads(l)
        if l_loaded['title']=='イギリス':
            new_f.write(l_loaded['text'])
            #1行だけでUKに関する記事が全てwriteされるので、ここでbreakする方がエレガント
!head 'article_UK'

{{redirect|UK}}
{{redirect|英国|春秋時代の諸侯国|英 (春秋)}}
{{Otheruses|ヨーロッパの国|長崎県・熊本県の郷土料理|いぎりす}}
{{基礎情報 国
|略名  =イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）


### ログ
pep8に従うと、open(path, mode='w')のmodeの両端はスペースを入れない  
ネットからデータセットを得たときは、コマンドラインで内容を確認しておくべし  
ex)  
!head -c で、バイト数基準で開く(1行に大変な量のデータがつまってるかもなので、バイト数基準で開け) 
json_keys() : jsonファイルの構造を教えてくれるコード

### 21.カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [4]:
article_UK = '/Users/seiya.k/workspace/100knock-2020/SeiyaKikuchi/chapter03/article_UK'

In [5]:
import re
with open(article_UK) as f:
    # 巨大なfileをreadするとき、readは使いたくない
    # でも、findallを使うには文字列にしなきゃいけない...
    # やっぱりre.searchを使う方がいいかもね
    article = f.read()
    category_names = re.findall(r'\[\[Category:.*\]\]', article)
[n for n in category_names]

['[[Category:イギリス|*]]',
 '[[Category:イギリス連邦加盟国]]',
 '[[Category:英連邦王国|*]]',
 '[[Category:G8加盟国]]',
 '[[Category:欧州連合加盟国|元]]',
 '[[Category:海洋国家]]',
 '[[Category:現存する君主国]]',
 '[[Category:島国]]',
 '[[Category:1801年に成立した国家・領域]]']

### 正規表現をコンパイルして、オブジェクトにして持っておく利点
いちいち正規表現にマッチするかどうかを問い合わせる必要がなくなるから、処理が軽くなる  
しかしコードの行数は多くなる

### 便利な文字列メソッド startswith を使う

### 正規表現を使わないで済ませたい

In [6]:
if '[Category' in line:

SyntaxError: unexpected EOF while parsing (<ipython-input-6-3989c8567959>, line 1)

### 22.カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [7]:
import re
with open(article_UK) as f:
    article = f.read()
    category_names = re.findall(r'\[\[Category:(.*)\]\]', article)
[n for n in category_names]

['イギリス|*',
 'イギリス連邦加盟国',
 '英連邦王国|*',
 'G8加盟国',
 '欧州連合加盟国|元',
 '海洋国家',
 '現存する君主国',
 '島国',
 '1801年に成立した国家・領域']

### グループのネーミング
グループに名前を付けておくとコードの可読性上昇  
[Category:(< category >.*)]

### wikiの公式ドキュメントからmarkupの規則を確認すること
例えば[カテゴリ：カテゴリ名]という表記も公式では許されている、という盲点  
やり直しだっ!!

### 23.セクション構造
記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [8]:
import re
with open(article_UK) as f:
    article = f.read()
    section_names = re.findall(r'(={2,4})([^=]*)={2,4}', article)
for n in section_names:
    print(f'{n[1]}:{len(n[0])-1}')

国名:1
歴史:1
地理:1
主要都市:2
気候:2
政治:1
元首:2
法:2
内政:2
地方行政区分:2
外交・軍事:2
経済:1
鉱業:2
農業:2
貿易:2
不動産:2
エネルギー政策:2
通貨:2
企業:2
通信:3
交通:1
道路:2
鉄道:2
海運:2
航空:2
科学技術:1
国民:1
言語:2
宗教:2
婚姻:2
移住:2
教育:2
医療:2
文化:1
食文化:2
文学:2
哲学:2
音楽:2
ポピュラー音楽:3
映画:2
コメディ:2
国花:2
世界遺産:2
祝祭日:2
スポーツ:2
サッカー:3
クリケット:3
競馬:3
モータースポーツ:3
野球:3
 カーリング :3
 自転車競技 :3
脚注:1
関連項目:1
外部リンク:1


### 24.ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [9]:
#fileへのlinkの記法　--> [[:File:Example.jpg]]　または　[[ファイル:tst.png]]
import re
with open(article_UK) as f:
    article = f.read()
    fname_pattern = re.compile(r'File|ファイル:([^\[\]]*?)\|')
    file_names = re.findall(fname_pattern, article)
[n for n in file_names][:10]

['Royal Coat of Arms of the United Kingdom.svg',
 'Descriptio Prime Tabulae Europae.jpg',
 "Lenepveu, Jeanne d'Arc au siège d'Orléans.jpg",
 'London.bankofengland.arp.jpg',
 'Battle of Waterloo 1815.PNG',
 'Uk topo en.jpg',
 'BenNevis2005.jpg',
 'Population density UK 2011 census.png',
 '2019 Greenwich Peninsula & Canary Wharf.jpg',
 'Birmingham Skyline from Edgbaston Cricket Ground crop.jpg']

#### 拡張子を手掛かりに...
#### ゼロ幅マッチ
#### finditerの利用

### 25.テンプレートの抽出
# findallを使わず書き直し
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [10]:
import pickle
def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data

In [11]:
import re
with open(article_UK) as f:
    article = f.read()

    temp_pattern = re.compile(r'^\{\{基礎情報 国\n\|(.*?)^\}\}$', re.MULTILINE+re.DOTALL)
    temp = re.findall(temp_pattern, article)[0]
    
    field_dict = dict()
    for field in temp.split("\n|"):
        #print(field)
        field = '\|' + field
        field_pattern = re.compile(r'\|(.*?)=(.*)', re.DOTALL)
        fields = re.findall(field_pattern, field)
        field_dict[fields[0][0].strip()] = fields[0][1].strip()
    pickle_dump(field_dict, 'dict_Q25')
[d for d in field_dict.items()][:5]

[('略名', 'イギリス'),
 ('日本語国名', 'グレートブリテン及び北アイルランド連合王国'),
 ('公式国名',
  '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>'),
 ('国旗画像', 'Flag of the United Kingdom.svg'),
 ('国章画像',
  '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]')]

### 公式国名フィールドは複数行に渡っているため、\nをフィールド間の区切りにしてはいけない...。しかし正規表現に組み込むと次の行の処理に差し支える。ダメだ...やっぱりre.match使うしかないか...。

### 26.強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

In [12]:
dict_Q25 = "/Users/seiya.k/workspace/100knock-2020/SeiyaKikuchi/chapter03/dict_Q25"

In [13]:
import re
dic = pickle_load(dict_Q25)

for k, v in dic.items():
    dic[k] = re.sub('\'{2,3}|\'{5}', '', v)
pickle_dump(dic, 'dict_Q26')
[d for d in dic.items()][7]

('国歌',
 '[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />神よ女王を護り賜え<br />{{center|[[ファイル:United States Navy Band - God Save the Queen.ogg]]}}')

##### itertoolsの便利ツールの内容を大体知っておくこと

# 27.内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．

In [14]:
dict_Q26 = "/Users/seiya.k/workspace/100knock-2020/SeiyaKikuchi/chapter03/dict_Q26"

In [15]:
import re
dic = pickle_load(dict_Q26)

for k, v in dic.items():
    inner_link_markup = re.compile('\[\[([^\#^\|]*?)\#??([^|^#]*?)\|??([^|]*?)\]\]')
    dic[k] = re.sub(inner_link_markup, r'\1\2\3', v)

pickle_dump(dic, 'dict_Q27')
[d for d in dic.items()][10:20]

[('公用語', '英語'),
 ('首都', 'ロンドン（事実上）'),
 ('最大都市', 'ロンドン'),
 ('元首等肩書', 'イギリスの君主女王'),
 ('元首等氏名', 'エリザベス2世'),
 ('首相等肩書', 'イギリスの首相首相'),
 ('首相等氏名', 'ボリス・ジョンソン'),
 ('他元首等肩書1', '貴族院 (イギリス)貴族院議長'),
 ('他元首等氏名1', ':en:Norman Fowler, Baron Fowlerノーマン・ファウラー'),
 ('他元首等肩書2', '庶民院 (イギリス)庶民院議長')]

startswithを使えばファイルmarkupと区別して削除できる  
否定的先読みを使用しても良い

def remove_link(text:str)->str:  
関数の頭で入力値と戻り値を明示的に示すと可読性高い。型が一致していないとエラーが出る...とかはしてくれないので、人間のための記述

# 28.MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [16]:
dict_Q27 = "/Users/seiya.k/workspace/100knock-2020/SeiyaKikuchi/chapter03/dict_Q27"

In [17]:
import re
dic = pickle_load(dict_Q27)

for k, v in dic.items():
    #外部リンク
    outer_link_markup = re.compile('\[(http:.*?)\]')
    v = re.sub(outer_link_markup, r'\1', v)
    
    #脚注
    #1つ目と3つ目の正規表現の処理がかぶっている
    #目についたものから処理していくと機能がかぶるから、コード全体を見渡す気構えが必要
    ref_markup = re.compile('<ref>')
    v = re.sub(ref_markup, r'', v)
    ref_front_markup = re.compile('<\/ref>')
    v = re.sub(ref_front_markup, r'', v)
    ref_middle_markup = re.compile('<ref .*?>')
    v = re.sub(ref_middle_markup, r'', v)
    
    #改行タグ
    br_tag = re.compile('<br \/>')
    v = re.sub(br_tag, r'', v)
    
    #言語テンプレート
    lang_temp = re.compile('\{\{lang\|.*?\|(.*?)\}\}')
    v = re.sub(lang_temp, r'\1', v)
    
    #引用テンプレート
    cite_temp = re.compile('\{\{(.*?)\}\}', re.DOTALL)
    v = re.sub(cite_temp, r'\1', v)
    
    #_markup = re.compile('')
    #v = re.sub(_markup, r'', v)
    
    dic[k] = v

pickle_dump(dic, 'dict_Q28')
[d for d in dic.items()]

[('略名', 'イギリス'),
 ('日本語国名', 'グレートブリテン及び北アイルランド連合王国'),
 ('公式国名',
  'United Kingdom of Great Britain and Northern Ireland英語以外での正式国名:\n*An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath（スコットランド・ゲール語）\n*Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon（ウェールズ語）\n*Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann（アイルランド語）\n*An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh（コーンウォール語）\n*Unitit Kinrick o Great Breetain an Northren Ireland（スコットランド語）\n**Claught Kängrick o Docht Brätain an Norlin Airlann、Unitet Kängdom o Great Brittain an Norlin Airlann（アルスター・スコットランド語）'),
 ('国旗画像', 'Flag of the United Kingdom.svg'),
 ('国章画像',
  '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]'),
 ('国章リンク', '（イギリスの国章国章）'),
 ('標語', 'Dieu et mon droit（フランス語:Dieu et mon droit神と我が権利）'),
 ('国歌',
  '[[女王陛下万歳|God Save the Queen]]en icon神よ女王を護り賜えcenter|ファイル:United States Navy Band - God Save the Queen.ogg'),
 ('地図画像', 'Europe-UK.svg'),
 ('位置画像',
  'United Kingdom (+overseas terr

### compileはfor文の外でやりましょう

# 29.国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [18]:
dict_Q28 = "/Users/seiya.k/workspace/100knock-2020/SeiyaKikuchi/chapter03/dict_Q28"

In [23]:
import json
import urllib.parse
import urllib.request

dic = pickle_load(dict_Q28)

request_url="https://www.mediawiki.org/w/api.php?"\
            + 'action=query'\
            + '&titles=File:' + urllib.parse.quote(dic['国旗画像'])\
            + '&format=json'\
            + '&prop=imageinfo'\
            + '&iiprop=url'
response=urllib.request.urlopen(request_url)
response_data=json.loads(response.read())
url=response_data['query']['pages']['-1']['imageinfo'][0]['url']
print(url)

{'continue': {'iistart': '2019-09-10T16:52:58Z', 'continue': '||'}, 'query': {'pages': {'-1': {'ns': 6, 'title': 'File:Flag of the United Kingdom.svg', 'missing': '', 'known': '', 'imagerepository': 'shared', 'imageinfo': [{'url': 'https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg', 'descriptionurl': 'https://commons.wikimedia.org/wiki/File:Flag_of_the_United_Kingdom.svg', 'descriptionshorturl': 'https://commons.wikimedia.org/w/index.php?curid=347935'}]}}}}
https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg


### 尾中くんのコードを手本に書き直せオラ!!
いろいろ甘い...so sweetだ

assert文を使ってみなさい

オブジェクト志向で頑張り、実際の処理は関数をlistにして順に適用する(=>t_takahashiさんを参考に)